# Getting started with MemsArray objects
® *Copyright Bimea 2024-2025*

The `MemsArray` class is the base class for building virtual antennas. 
This Notebook shows how you can use the `MemsArray` interface to run simple antennas experiences.

In [ ]:
import time
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt

from megamicros.log import log
from megamicros.core.base import MemsArray

# set log level to INFO to get some informations (available levels are DEBUG, INFO, WARNING, ERROR, FATAL)
log.setLevel( "INFO" )

## Declare an antenna object

The ``MemsArray`` constructor needs no argument. Once created, a `MemsArray` object can be considered as a virtual antenna for which all parameters are set to their default value.

In [ ]:
antenna = MemsArray()

# Get some infos about the antenna as a dictionary
pprint( antenna.infos )


Activate the first 8 MEMS and generate signals

In [ ]:
# Activate the first 8 MEMS and generate 1 second acquisition
antenna.setActiveMems( list(range(8)) )
antenna.setDuration( 1 )          
antenna.run()
antenna.wait()

Getting data can be done by iterate through the `antenna` object provided the antenna had already received all the data:

In [ ]:
# Print frames stored in the queue
print(f"queue content : {antenna.queue_content} frames")

# Retrieve data from the queue
for data in antenna:
    print( f"data={data}" )

Once the frames have been retrieved, the queue is empty:

In [ ]:
# Print frames stored in the queue
print(f"queue content : {antenna.queue_content} frames")

Settings can be passed to the run() method as well:

In [ ]:
antenna.run(
    duration=1,                             # acquisition time in seconds 
    mems = antenna.available_mems,          # activate all available MEMs
    counter = False,                        # do not get counter channel
    datatype = 'int32',                     # set datatype to ìnt32 (default)
    sampling_frequency=50000                # set sampling frequency to 50kHz
)

antenna.wait()

In [ ]:
# Only set MEMS 0 and 10. All previous args remain the same:
antenna.run( 
    mems = [0, 10],
)

antenna.wait()

The previous method suggests that real time is not possible.
Actually, you can perform real time generation if you iterate before calling the `wait` method, just after the `run` call. 
The important point is that the `run()` call is non-blocking. As such it implies that the `wait()` method is mandatory and must not be forgotten: 

In [ ]:
log.setLevel( "WARNING" ) 

antenna.run( 
    duration=1,       # acquisition time in seconds
    mems=[0, 1],      # activate only MEMs 0 and 1
    counter=True,     # get counter channel 
)

# Get only the counter (first channel)
for data in antenna:
    print( f"Counter value: {data[0][0]}" )

antenna.wait()

# No more frames stored in the queue
print(f"queue content : {antenna.queue_content} frames")

Let us measure the acquisition time: 

In [ ]:
start_time = time.time()
antenna.run( 
    duration=1,       # acquisition time in seconds
    mems=[0, 1],      # activate only MEMs 0 and 1
    counter=True,     # get counter channel 
)
# Get only the counter (first channel)
counter = []
for data in antenna:
    counter.append(int(data[0][0]))
antenna.wait()
end_time = time.time()

print(f"Acquisition time: {end_time - start_time} seconds")

Note that the elapsed time reaches 2 seconds, whereas the requested acquisition time was 1 second.

The method, as reported above, includes, in addition to the acquisition time (1s):
* The timeout delay to leave the queue (default is 1 second).

As a consequence, the total elapsed time is 2 seconds.

## Add MEMS positions 

`MemsArray` objects can be considered as virtual antenna placed in some position in the space. 
MEMS positions are then defined relatively to the center of the antenna.

In the following example we build a circular 32 MEMS antenna with a diameter of 350mm

In [ ]:
import numpy as np
from megamicros.geometry import circle

# Define the antenna geometry
mems_number = 32
antenna_radius = 0.175
antenna_thickness = 0.0
antenna_angle_offset = 2 * np.pi / mems_number / 2

mems_position = np.array( circle( 
    points_number=mems_number, 
    radius=antenna_radius, 
    height=antenna_thickness, 
    angle_offset=antenna_angle_offset, 
    clockwise=True ) )

# Create the antenna
antenna = MemsArray()

antenna.setMemsPosition( mems_position )
print( "MEMS Position:", antenna.mems_position )

See the `04_getstarted_BMF` Notebook for information on how to perform beamforming with antennas. 